# 02d — Evaluate Cophenetics

In [1]:
from pathlib import Path
import pickle, math

ROOT=Path.cwd()
for c in (ROOT,*ROOT.parents):
    if (c/"outputs"/"upgma.pkl").exists(): ROOT=c; break
OUT=ROOT/"outputs"
up=pickle.load(open(OUT/"upgma.pkl","rb"))
sd=pickle.load(open(OUT/"similarity_distance.pkl","rb"))
langs=up["langs"]; merges=up["merges"]; Dist=sd["Dist"]; n=len(langs)
print("n:", n)

n: 16


### Step 1: Cophenetic correlation — how well does the tree fit the distances?

This cell measures how faithfully the dendrogram (from `merges`) preserves the original pairwise distances `Dist` using the **cophenetic correlation coefficient**.

What it does:
- Builds `parent` and `height` maps from `merges`, where each internal node `new` has merge height `h`.
- For each leaf `i`, `anc(i)` returns all ancestors up to the root with their heights.
- `lca_h(i, j)` finds the **lowest common ancestor (LCA)** of leaves `i` and `j` and returns its merge height — the **cophenetic distance** between `i` and `j`.
- Collects:
  - `X`: original distances `Dist[i][j]`
  - `Y`: cophenetic distances `lca_h(i, j)`
- Computes the Pearson correlation `r` between `X` and `Y`:
  \[
    r = \frac{\sum (X-\bar X)(Y-\bar Y)}{\sqrt{\sum (X-\bar X)^2}\;\sqrt{\sum (Y-\bar Y)^2}}
  \]
- Prints `r` (close to 1 means the hierarchical clustering structure matches the original distance geometry well).

Notes:
- Cophenetic distances are ultrametric by construction; large deviations from 1 suggest the data are far from ultrametric or linkage choice is mismatched.
- If multiple LCAs occur at the same height (ties), correlation can plateau even if the tree is conceptually good.

In [2]:
parent,height={},{}
for l,r,h,new in merges: parent[l]=new; parent[r]=new; height[new]=h

def anc(i):
    a={}
    while i in parent:
        p=parent[i]; a[p]=height[p]; i=p
    return a
A=[anc(i) for i in range(n)]

def lca_h(i,j):
    ai,aj=A[i],A[j]
    for node,h in ai.items():
        if node in aj: return h
    return merges[-1][2]

X=[]; Y=[]
for i in range(n):
    for j in range(i+1,n):
        X.append(Dist[i][j]); Y.append(lca_h(i,j))

mx=sum(X)/len(X); my=sum(Y)/len(Y)
num=sum((a-mx)*(b-my) for a,b in zip(X,Y))
den=(sum((a-mx)**2 for a in X)*sum((b-my)**2 for b in Y))**0.5 or 1e-12
r=num/den
print("Cophenetic correlation r =", round(r,3))

Cophenetic correlation r = 0.935
